# Call Options and Market Making

In [50]:
import os
import plotly.express as px
import plotly.graph_objects as go

import numpy as np
import pandas as pd

import src.payoffs as po
import src.black_scholes as bs
import src.option_hedger as oh
import src.data_loaders as dl
import src.plot_utils as pl

from src.paths import geom_brownian_path
import src.colnames as n

In [51]:
# Choose fixed values for Maturity, Strike and RiskFree rate
T = 1
K = 100
R = 0

# Choose some values for \sigma in percent (input 20 for 20%)
sigmas_prc = [5, 20, 60]

## General Descriptions

### Call Price

Under the Black and Scholes model (B&S), the price, at time $0$, of the call option of strin $K$ and maturity $T$ is :

$$C(K,T,r,S,\sigma) = S \mathcal{N}(d_1) - K e^{-rT}\mathcal{N}(d_2)$$

Where :

* $\mathcal{N}$ is the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) of the standard Gaussian distribution.
* $d_1 = ln(\frac{S_t}{K}) + \frac{(r + \sigma^2/2)T}{\sigma \sqrt{t}}$ 
* $d_2 = d_1 - \sigma \sqrt{T}$
* $T$ is the time to maturity (because we are at time $0$)
* $r$ is the risk-free rate
* $\sigma$ is the volatility of the B&S diffusion model
* $S$ is the token spot price

Here is a quick view of this price curve, as a function of $S$ :

In [74]:
K, SIGMA, T, r = 100, 0.2, 1, 0
xr = list(range(int(K/3), int(1.5*K)))
fig = pl.new_subplot()
fig.add_scatter(x=xr, y=[bs.call_price(SIGMA, K, T, r, x) for x in xr], name="Call Price")
fig.add_scatter(x=xr, y=[po.call(x, K) for x in xr], name="Call Payoff")
fig.update_layout(title_text="Call Price and Payoff")
fig.show()

### Zoom on Delta ($\Delta$)

$$\Delta = \frac{\partial C}{\partial S} = \mathcal{N}(d_1)$$

Where .

* $\Delta$ is the sensivity of the Call price to a small variation of the underlying token spot price $S$
* $\Delta$ depends on various parameters (token spot $\S$, token volatility $\sigma$, time to maturity, risk-free rate,...)
  * Usually, we only use "time subscript" to represent it's value: $\Delta_t$
* For a Call Option : 
  * $0 \leq \Delta_t \leq 1$
  * $\Delta$ increases with token spot price


In [53]:
sigmas = [s/100 for s in sigmas_prc]

def fun_delta(x, s):
    return bs.call_delta(s, K, T, R, x)

# generate delta curves
res = []
for x in range(int(K/10), 2 * K):
    res.append([x, *[fun_delta(x, s) for s in sigmas]])

deltas = pd.DataFrame(res)
deltas.columns = ["Spot", *sigmas_prc]
deltas.set_index( "Spot", inplace=True)

# display
title_template = "Delta curves depending on sigma [Fixed K={K}, r={R}, T={T}y]"
title_plot = title_template.format(K=K, R=R, T=T)
fig = px.line(deltas, title=title_plot).add_vline(x=K, line_width=2, line_dash="dash")
fig.show()

In [54]:
# generate delta/gamma curves
res = []
for x in range(int(K/2), int(1.6 * K)):
    for t in [0.05, 0.25, 1]:
        res.append([x, t, bs.call_delta(0.2, K, t, R, x), bs.call_gamma(0.2, K, t, R, x)])

rdf = pd.DataFrame(res)
rdf.columns = ["Splot", "Ttm", "Delta", "Gamma"]

In [55]:
px.line(rdf, x="Splot", y ="Delta", color="Ttm", title="Delta depending on time to maturity")

### Zoom on Gamma ($\Gamma$)

$$\Gamma = \frac{\partial^2 C}{\partial S^2} = \frac{1}{x\sigma\sqrt{T}}\mathcal{N}'(d1) > 0 $$

Where $\mathcal{N}'$ is the [density](https://en.wikipedia.org/wiki/Normal_distribution) of the standard Gaussian distribution.

* $\Gamma$ is the sensivity of the $\Delta$ to a small variation of the underlying token spot price $S$
  * If *positive* : this is the amount of token you have to buy to adjust your $\Delta$ position when price goes *up*.
  * If *negative* : this is the amount of token you have to buy to adjust your $\Delta$ position when price goes *down*.
* $\Gamma$ depends on various parameters (token spot $\S$, token volatility $\sigma$, time to maturity, risk-free rate,...)
  * Usually, we only use "time subscript" to represent it's value: $\Gamma_t$
* For a Call Option : 
  * $\Gamma$ is positive


In [56]:
sigmas = [s/100 for s in sigmas_prc]

# generate gamma curves
res = []
for x in range(int(K/10), 2 * K):
    res.append([x, *[bs.call_gamma(s, K, T, R, x) for s in sigmas]])

gammas = pd.DataFrame(res)
gammas.columns = ["Spot", *sigmas_prc]
gammas.set_index( "Spot", inplace=True)

# display
title_plot = "Gamma curves depending on sigma [Fixed K={K}, r={R}, T={T}y]".format(K=K, R=R, T=T)
px.line(gammas, title=title_plot).add_vline(x=K, line_width=2, line_dash="dash").show()

In [57]:
px.line(rdf, x="Splot", y ="Gamma", color="Ttm", title="Gamma depending on time to maturity")

## Replication in Action

### Call Price, and Replication/Hedging Portfolio

Suppose a nice world where : 

1. Our token spot price $S_t$ behaves like a B&S model with a fixed volatility $\sigma$
2. We can trade long/short, without limitation and impact on the maket

Let's put in place a portfolio that will hold $\Delta_t$ amount of token, at each time step :

* We will simulate a token spot price trajectory (daily, until the maturity of the call option)
* We will build a *hedging portfolio*, called $V$, by :
  * adjusting our quantity of token to match $\Delta_t$, every day

We want to **see** that this portfolio is linked to the **theoretical** value of the call option.

In [58]:
# Let's choose our parameters

# --- for the call option
T = 1               # time to maturity of the call option, in years
K = 100             # strike of the call option

# --- for the token price B&S model
SIGMA = 0.10        # volatility 0.2 = 20%
S0 = 100            # initial token price
MU = 0.0            # token spot price drift, 0.05=5%

# --- for the market (can be left at 0)
R = 0               # risk free rate

# --- for the random generator 😀
SEED = 201


In [59]:
path, dt = geom_brownian_path(S0, MU, SIGMA, int(T*365), 1, SEED)
# px.line(path).show()

# replicate_call(sigma, K, T, r, hedge_threshs, hedge_win, path, dt, store=False)
states = oh.replicate_call(SIGMA, K, T, R, hedge_threshs=0.5, hedge_win=1*dt, path=path, dt=dt, store=True)

sdf = pd.DataFrame(states)
sdf[n.Option] = sdf.apply(lambda x: bs.call_price(SIGMA, K, x[n.Ttm], R, x[n.Price]), axis=1)
sdf[n.OptionHedged] = sdf[n.Option] - sdf[n.Valo]
# px.line(sdf).show()

# Create figure with secondary y-axis
fig = pl.new_subplot()

visibles = [n.Option, n.Valo, n.OptionHedged]

def add_line(col, second_axis):
    visi = True if col in visibles else "legendonly"
    fig.add_trace(
        go.Scatter(x=sdf.index, y=sdf[col], 
            name="{0} ({1})".format(col, "right" if second_axis else "left"), visible=visi), 
        secondary_y=second_axis)

# Add traces
fig.add_trace(go.Bar(x=sdf.index, y=sdf[n.Delta], name="{0} ({1})".format(n.Delta, "left"), visible="legendonly"), secondary_y=False)
[add_line(col, True) for col in [n.Price, n.Option, n.Valo, n.OptionHedged]]
# [add_line(col, True) for col in [n.Delta]]

fig.update_layout(title_text="Option hedging", yaxis=dict(side="right"), yaxis2=dict(side="left"))
fig.show()

**Legend explanations**

* *Price*: token spot price over the days ($S_t$)
* *Option*: theoretical value of the Call option $C_t$ (B&S formula, with $\sigma$, etc...)
* *Repli*: value of our hedging portfolio $V_t$ over the days
* *OptionHedged*: Option - Repli
* *Delta*: quantity of token hold by $V_t$ every day (built to be the theoretical B&S $\Delta_t$)

**Observations**

1. At step 1 (or time 0), Call price $C_0$ comes from a probabilistic computation => it does not depend on the (randomly generated) path
2. At step 365 (= time 1), Call price is not model dependent anymore, it's value is $C_T=max(S_T - K, 0)$.
3. We observe that the spread $C_t - V_t$ is constant over the full time window, equals to $C_0$
4. We feel that the hedging portfolio $V$ only miss this $C_0$ amount of cash to perfectly replicate the call option at maturity (independently from the path)
   * Feel free to modify MU or to change the seed to see that the spread has always the same value 

#### Conclusion

**In a simulated market, the call price $C_0$ is the initial cost of it's hedging/replicating portfolio $V_t$.**


### Replication Risks

Hedging a Call option is easy in a perfectly theoretical world, when token spot prices follow a nice and steady B&S continuous market model.

Let's observe what happens in concrete examples to form an intuition of the replication risks.

I took a fresh exemple of [SPX](https://en.wikipedia.org/wiki/S%26P_500) (SP500, the most famous equity index) historical data over the last 5 years. 

For volatility estimation, let's choose a time window to compute rolling historical standard deviation on [logarithmic returns](https://en.wikipedia.org/wiki/Rate_of_return#Logarithmic_or_continuously_compounded_return).

In [60]:
# read historical SPX data
filepath = os.path.join(
    "data", "market", "2017-05-23_2022-05-20_SPX_nasdaq_com.csv")
df = dl.read_csv_nasdaq_com(filepath)
df = df[[n.Date, "C"]]
df.columns = [n.Date, n.Price]
df = df.set_index(n.Date).sort_index()

In [61]:
STD_WIN = 120

In [62]:
# Compute historical volatility
df[n.Dlog] = np.diff(np.log(df[n.Price].shift(2)), prepend=np.nan)
df[n.Hvol] = df[n.Dlog].rolling(STD_WIN).std().multiply(np.sqrt(365)).shift(2)

# Display SPX and Historical Volatility
fig = pl.new_subplot()
pl.add_serie(fig, df, n.Price)
pl.add_serie(fig, df, n.Dlog, go.Bar, True, False)
pl.add_serie(fig, df, n.Hvol, second_axis=True)
fig.add_vrect(x0="2020-01-02", x1="2020-12-31",
              line_width=0, fillcolor="green", opacity=0.2)
fig.add_vrect(x0="2021-01-02", x1="2021-12-31",
              line_width=0, fillcolor="red", opacity=0.2)
fig.show()

#### Bull case (2020), Bear case (2021)

Pick a year on the following cell:

In [75]:
YEAR = 2020

In [76]:
# Hedging a long Call position
R = 0

# Compute option prices and replication portfolio
df_sub = df.loc[df.index.year == YEAR]
date_from, date_to = df_sub.iloc[[0, -1]].index

K = S0 = df_sub.iloc[0][n.Price]
SIGMA = df_sub.iloc[0][n.Hvol]
T = (date_to - date_from).days/365
dt = T/len(df_sub)

# hedges_dlog = np.exp([0.05 * i for i in range(-40,40)])
# states = replicate_call(SIGMA, K, T, R, list(hedges_dlog * S0), 100*dt, path, dt, store=True)
states = oh.replicate_call(SIGMA, K, T, R, 0.5, dt,
                        df_sub[n.Price].to_list(), dt=T/len(df_sub), store=True)

sdf = pd.DataFrame(states)
sdf[n.Option] = sdf.apply(lambda x: bs.call_price(
    SIGMA, K, x[n.Ttm], R, x[n.Price]), axis=1)
sdf[n.OptionHedged] = sdf[n.Option] - sdf[n.Valo]
sdf[n.Hvol] = df_sub[n.Hvol].to_numpy()
sdf["Over"] = (0.0+(df_sub[n.Dlog].abs() * np.sqrt(365)
               > df_sub[n.Hvol])).to_numpy()

fig = pl.new_subplot()
pl.add_serie(fig, sdf, n.Price, displayed=False)
pl.add_serie(fig, sdf, n.Valo)
pl.add_serie(fig, sdf, n.Option)
pl.add_serie(fig, sdf, n.OptionHedged)
pl.add_serie(fig, sdf, n.Delta, displayed=False, second_axis=True)
pl.add_serie(fig, sdf, n.Cash, displayed=False, second_axis=True)
pl.add_serie(fig, sdf, n.Hvol, second_axis=True, displayed=False)
fig.update_layout(title_text="Study of SPX Call K={0} & SIGMA={1:.0f}% for year {2}.".format(
    int(K), 100*SIGMA, YEAR))
fig.show()


**Comments**

* Case 2020 : with the sharp market drop, realized volatilty spiked. At the same time we see that the Hedged Call position increases in value.
* Case 2021 : daily dlog returns realized below the snapshoted $\sigma$. We can see the decreasing historical volatility. It seems to have a negative impact on the hedged call value.

#### Tracking error general formula



## Hedge a Call $\implies$ run a Market Making strategy

### Increasing $\Delta$ (positive $\Gamma$)

1. To hedge a call, you need to be short (= to be seller) of $\Delta_t$ tokens at every time step.
2. But we know that for $\Delta_t$ increases with $S_t$, so:
   * When token price is up $\implies$ you need to sell some
   * When token price is down $\implies$ you need to buy some
3. This is exactly what a *Market Making* strategy is.

**In terms of $\Gamma$**

We can say that hedging a positive $\Gamma$ option (this is true for the call) is a market making strategy. 

### Observe the market making equivalent of SPX hedged call

Previous SPX scenario will be hedged every X%, by a simple market making strategy (buy/sell limit orders).

Pick your hedging range in the following cell:

In [77]:
HEDGE_RANGE = 0.025

In [78]:
hedges_dlog = np.exp([HEDGE_RANGE * i for i in range(-int(0.9/HEDGE_RANGE),int(3/HEDGE_RANGE))])
states = oh.replicate_call(SIGMA, K, T, R, list(hedges_dlog * S0), 100*dt, df_sub[n.Price].to_list(), dt, store=True)

sdf = pd.DataFrame(states)
sdf[n.Option] = sdf.apply(lambda x: bs.call_price(
    SIGMA, K, x[n.Ttm], R, x[n.Price]), axis=1)
sdf[n.OptionHedged] = sdf[n.Option] - sdf[n.Valo]
sdf[n.Hvol] = df_sub[n.Hvol].to_numpy()
sdf["Over"] = (0.0+(df_sub[n.Dlog].abs() * np.sqrt(365)
               > df_sub[n.Hvol])).to_numpy()

fig = pl.new_subplot()
pl.add_serie(fig, sdf, n.Price)
pl.add_serie(fig, sdf, n.LimBid)
pl.add_serie(fig, sdf, n.LimAsk)

pl.add_serie(fig, sdf, n.Valo, displayed=False)
pl.add_serie(fig, sdf, n.Option, displayed=False)
pl.add_serie(fig, sdf, n.OptionHedged, displayed=False)
pl.add_serie(fig, sdf, n.Delta, displayed=False, second_axis=True)
pl.add_serie(fig, sdf, n.Cash, displayed=False, second_axis=True)
pl.add_serie(fig, sdf, n.Hvol, second_axis=True, displayed=False)
fig.update_layout(title_text="Study of SPX Call K={0} & SIGMA={1:.0f}% for year {2}, hedge every {3:.2f}%".format(
    int(K), 100*SIGMA, YEAR, HEDGE_RANGE*100))
fig.show()